Replicate EV/REV plots for Octopamine/Rolipram sessions previously generated by Jahangir.

In [2]:
# Import all the functions you need
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
import os
from pathlib import Path
import platform

# Comment here to add in for either mac or linux computer
if platform.system() == "Linux":   # for linux
    sys.path.extend(['/data/GitHub/NeuroPy']) 
    sys.path.extend(['/data/GitHub/TraceFC/'])
elif platform.system() == "Darwin":
    sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/NeuroPy'])
    sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/TraceFC'])
elif platform.system() == "Windows":
    print('No windows repositories defined')

from neuropy.core.neurons import Neurons
from neuropy.analyses.reactivation import ExplainedVariance
from neuropy.core.epoch import Epoch
from neuropy.core.session import ProcessData

plot_save_dir = Path('/home/nkinsky/Dropbox (University of Michigan)/Shared_Folders/KD/Grants/R01_octopamine_rollipram/Nat_plots')

/home/nkinsky/miniforge3/envs/NeuroPy/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


### Get session info

In [3]:
sesh_csv = "/data3/Octopamine_Rolipram/Recording_Rats/OCT_ROL_sessions.csv"
sesh_df = pd.read_csv(sesh_csv, header=0)
sesh_df['session'] = pd.Categorical(sesh_df['session'], ['PBS', 'SAL', 'ROL', 'OCT'])
sesh_df = sesh_df.sort_values(["Animal", "session"]).reset_index().drop(columns = "index")
sesh_df

,Animal,session,date,directory,processed
0,A11,PBS,2019_12_02,/data3/Octopamine_Rolipram/Recording_Rats/A11/...,Y
1,A11,ROL,2019_12_12,/data3/Octopamine_Rolipram/Recording_Rats/A11/...,Y
2,A11,OCT,2019_11_30,/data3/Octopamine_Rolipram/Recording_Rats/A11/...,Y
3,A14,PBS,2020_02_23,/data3/Octopamine_Rolipram/Recording_Rats/A14/...,Y
4,A14,ROL,2020_02_26,/data3/Octopamine_Rolipram/Recording_Rats/A14/...,Y
5,A14,OCT,2020_02_21,/data3/Octopamine_Rolipram/Recording_Rats/A14/...,Y
6,BG,SAL,2019_10_21,/data3/Octopamine_Rolipram/Recording_Rats/BG/B...,Y
7,BG,ROL,2019_10_19,/data3/Octopamine_Rolipram/Recording_Rats/BG/B...,Y
8,BG,OCT,None,None,N
9,BGU,SAL,2021_08_11,/data3/Octopamine_Rolipram/Recording_Rats/BGU/...,Y


In [8]:
df_use = sesh_df[(sesh_df.Animal == "A14") & (sesh_df.session == "OCT")]
working_dir = Path(df_use.directory.values[0])
df_use

,Animal,session,date,directory,processed
5,A14,OCT,2020_02_21,/data3/Octopamine_Rolipram/Recording_Rats/A14/...,Y


In [35]:
pd.read_csv(shank_dir / "cluster_info.tsv", delimiter="\t")

,id,amp,ch,depth,fr,group,n_spikes,purity,sh
0,0,319.456482,0,240.0,4.902165,NaN,239082,0.571536,1.0
1,1,285.207703,14,30.0,2.420695,NaN,118059,0.817290,1.0
2,2,428.735291,0,240.0,2.951465,NaN,143945,0.294215,1.0
3,3,2393.944336,0,240.0,0.047282,noise,2306,0.933836,1.0
4,4,2151.953125,0,240.0,0.146727,noise,7156,0.935637,1.0
5,5,355.698853,2,210.0,6.435790,noise,313878,0.845830,1.0
6,6,358.271301,3,195.0,3.579444,NaN,174572,0.597005,1.0
7,7,321.801392,4,180.0,1.599772,noise,78022,0.846750,1.0
8,8,373.377838,1,225.0,4.018437,NaN,195982,0.593127,1.0
9,9,385.220947,1,225.0,5.736272,NaN,279762,0.483115,1.0


In [26]:
from neuropy.io.phyio import PhyIO
shank_num = 1
shank_dir = working_dir / "spyk-circ" / f"shank{shank_num}"
phy_shank = PhyIO(shank_dir, include_groups=["noise", np.nan])
phy_shank.spiketrains

array([array([2.20193333e+00, 3.54606667e+00, 5.36070000e+00, ...,
              4.87688578e+04, 4.87699292e+04, 4.87700500e+04])    ,
       array([3.24043333e+00, 5.87736667e+00, 5.96926667e+00, ...,
              4.87677353e+04, 4.87688251e+04, 4.87705101e+04])    ,
       array([  105.84496667,   107.22933333,   107.24033333, ...,
              48770.63426667, 48770.63576667, 48770.64006667])    ,
       array([  281.59986667,   281.96006667,   317.2995    , ...,
              48689.27176667, 48689.28213333, 48689.3471    ])    ,
       array([  122.4139    ,   159.24016667,   176.7658    , ...,
              48715.2945    , 48715.47523333, 48770.63426667])    ,
       array([6.24166667e-01, 8.48266667e-01, 8.72933333e-01, ...,
              4.87701624e+04, 4.87702024e+04, 4.87703225e+04])    ,
       array([2.87666667e-01, 4.73866667e-01, 5.76823333e+00, ...,
              4.87685048e+04, 4.87701685e+04, 4.87706343e+04])    ,
       array([1.61266667e-01, 7.40800000e-01, 1.1367000

In [37]:
phy_shank.cluster_info["group"].isin([np.nan])

0      True
1      True
2      True
3     False
4     False
5     False
6      True
7     False
8      True
9      True
10    False
11     True
12     True
13    False
14    False
15    False
16    False
Name: group, dtype: bool

### Loop through and update everything

In [172]:
for sesh_name in ["OCT", "PBS", "ROL"]:
    df_use = sesh_df[(sesh_df.Animal == "A11") & (sesh_df.session == sesh_name)]
    working_dir = Path(df_use.directory.values[0])
    
    # Get XML file name
    xmlfile = sorted(working_dir.glob("*.xml"))[0]

    # Build up epochs files
    blocks_dict = np.load(sorted(working_dir.glob("**/*_epochs.npy"))[0], allow_pickle=True).item()
    blocks_dict
    starts = [block[0] for block in blocks_dict.values()]
    stops = [block[1] for block in blocks_dict.values()]
    labels = list(blocks_dict.keys())
    epoch_df = pd.DataFrame({"start": starts, "stop": stops, "label": labels})
    epoch_df.loc[epoch_df.label == "run", "label"] = "maze"
    epoch_df.loc[epoch_df.label == "rs", "label"] = "post"
    epochs = Epoch(epochs=epoch_df)
    epochs.save(xmlfile.with_suffix(".epoch.npy"))

    # Load in basics
    basics = np.load(sorted(working_dir.glob("archive/*_basics.npy"))[0], allow_pickle=True).item()

    # Load in old-style neurons file
    neurons_old = Neurons.from_file(sorted(working_dir.glob("**/*_spikes.npy*"))[0])

    # Build up waveforms - make it zero everywhere except for the shank in question
    basics = np.load(sorted(working_dir.glob("archive/*_basics.npy"))[0], allow_pickle=True).item()
    nchan = basics['nChans']
    nneurons = len(neurons_old['templates'])
    nbins = neurons_old['templates'][0].shape[1]
    waveforms = []
    for i in range(nneurons):
        waveform = np.zeros((nchan, nbins))
        shank = neurons_old['info'].loc[i, "shank"].astype(int)
        shank_channels = basics['channelgroups'][shank - 1]
        try:
            waveform[np.min(shank_channels): (np.max(shank_channels) + 1)] = neurons_old['templates'][i]
        except ValueError:
            nchan_on_shank = neurons_old['templates'][i].shape[0]
            waveform[np.min(shank_channels): (np.min(shank_channels) + nchan_on_shank)]
            print(f"Channel # mismatch for unit {i} on shank {shank}! Be sure to check. Hacking it for now.")
            
        waveforms.append(waveform)

    # Build up new neurons file and save
    neurons_new = Neurons(neurons_old["times"], t_stop = epochs['post'].stops[0], neuron_type=neurons_old["info"].group.values,
                          waveforms=np.array(waveforms))
    neurons_new.save(xmlfile.with_suffix(".neurons.npy"))

/data3/Octopamine_Rolipram/Recording_Rats/A11/A11_2019-11-30_SDOCT/A11_2019-11-30_SDOCT.epoch.npy saved
/data3/Octopamine_Rolipram/Recording_Rats/A11/A11_2019-11-30_SDOCT/A11_2019-11-30_SDOCT.neurons.npy saved
/data3/Octopamine_Rolipram/Recording_Rats/A11/A11_2019-12-02_SDPBS/A11_2019-12-02_SDPBS.epoch.npy saved
Channel # mismatch for unit 3 on shank 2! Be sure to check. Hacking it for now.
Channel # mismatch for unit 4 on shank 2! Be sure to check. Hacking it for now.
/data3/Octopamine_Rolipram/Recording_Rats/A11/A11_2019-12-02_SDPBS/A11_2019-12-02_SDPBS.neurons.npy saved
/data3/Octopamine_Rolipram/Recording_Rats/A11/A11_2019-12-12_SDROL/A11_2019-12-12_SDROL.epoch.npy saved
/data3/Octopamine_Rolipram/Recording_Rats/A11/A11_2019-12-12_SDROL/A11_2019-12-12_SDROL.neurons.npy saved
